# Hyperparameter Optimization

This notebook is a tutorial on using AmpOpt to tune an amptorch model hyperparameters.

Before starting this notebook, please make sure that you've followed all the steps in [SETUP.md](../docs/SETUP.md).

In [1]:
import ampopt
from ampopt.utils import format_params
from ampopt.study import get_study

# 1. Create MySQL Port

In order to run hyperparameter tuning jobs, we need a MySQL port.

update ".env" if ssh is required include the last five argument in env

```
MYSQL_USERNAME=
MYSQL_PASSWORD=
HPOPT_DB=
MYSQL_HOSTNAME=

MYSQL_PORT=
SSH_HOST=
SSH_USER=
SSH_PASS=
SSH_PORT=
```

## 2. Preprocessing

AmpOpt requires data to be preprocessed using the preferred fingerprinting scheme and preprocessing pipeline, and saved in LMDB format, before hyperparameter optimization. This saves a lot of work being wasted performing the featurization for every optimization trial.

With AmpOpt, preprocessing and saving to LMDB is as easy as:

In [ ]:
ampopt.preprocess("../data/oc20_3k_train.traj", "../data/oc20_300_test.traj")

The data should be readable by either `ase.io.Trajectory` or `ase.io.read`. 

If you have several files, the first will be used to fit the transformers (e.g. for feature scaling). This prevents data leakage.

## 3. Running an Individual Training Job

Before we launch into running hyperparameter tuning jobs, let's train an individual model and evaluate it to get a (poor) baseline.

In [ ]:
ampopt.eval_score(
    epochs=10,
    train_fname="../data/oc20_3k_train.lmdb",
    valid_fname="../data/oc20_300_test.traj",
    dropout_rate=0.,
    lr=1e-3,
    gamma=1.,
    num_nodes=5,
    num_layers=5,
)

The performance of this model is poor, but that's to be expected: we only trained it for 10 epochs. We'll improve this score in the next section.

## 4. Running Single Tuning Jobs

Let's first run a single tuning job to try and find the optimal number of layers and number of nodes per layer when training for just 10 epochs.

We only need to supply a single dataset; amptorch will split 10% of the data off as a validation set.

The `study` argument can be anything, though we should be careful not to name this study the same as a previous study. It's how we'll later retrieve the study to perform analysis.

For `params`, we can pass any of the following hyperparameters:

- Learnable Parameters:
    - `num_layers`, the number of layers of the neural network
    - `num_nodes`, the number of nodes per layer
    - `dropout_rate`, the rate of [dropout](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/) during training
    - `lr`, the learning rate for gradient descent
    - `gamma`, the decay parameter for the learning rate.
- Non-Learnable Parameters:
    - `step_size`, the number of epochs after which the learning rate decreases by `gamma`
    - `batch_size`, the size of minibatches for gradient descent

Any learnable parameter not fixed in the `params` argument will be learned during hyperparameter optimization. Any non-learnable parameter will be given a default value.

The learnable and non-learnable parameters, as well as default values in the amptorch config, are specified in `src/ampopt/train.py`. Feel free to tweak this.

In [3]:
ampopt.tune(
    study="tutorial1",
    trials=10,
    epochs=10,
    data="../data/oc20_3k_train.lmdb",
    params=format_params(
        dropout_rate=0.0,
        gamma=1.0,
    ),
)

Running hyperparam tuning with:
 - study_name: tutorial1
 - dataset: ../data/oc20_3k_train.lmdb
 - n_trials: 2
 - sampler: CmaEs
 - pruner: Hyperband
 - num epochs: 2


[I 2022-11-07 01:45:10,920] Using an existing study with name 'tutorial1' instead of creating a new one.


 - params:
   - dropout_rate: 0.0
   - gamma: 1.0
CmaEsSampler


loading from C:\Users\ryanc\Desktop\Student Files\VIP\Alex-Repo\bdqm-hyperparam-tuning\data\oc20_3k_train.lmdb:  15%|▏|

Results saved to ./checkpoints\2022-11-07-01-45-11-de3b7ce5-7a51-4816-b6d4-a3dca5426142


...

[I 2022-11-07 01:45:13,338] Trial 1 finished with value: 916.1633675160737 and parameters: {'num_layers': 11, 'num_nodes': 15, 'lr': 0.031263973163183786}. Best is trial 1 with value: 916.163.
loading from C:\Users\ryanc\Desktop\Student Files\VIP\Alex-Repo\bdqm-hyperparam-tuning\data\oc20_3k_train.lmdb:  15%|▏|

Results saved to ./checkpoints\2022-11-07-01-45-13-ce1f3761-b136-40a3-89c2-32281b8ad1e0


loading from C:\Users\ryanc\Desktop\Student Files\VIP\Alex-Repo\bdqm-hyperparam-tuning\data\oc20_3k_train.lmdb: 100%|█|


Loading dataset: 3000 images
Use Xavier initialization
Loading model: 5207 parameters
Loading skorch trainer
Training completed in 0.9900028705596924s


[I 2022-11-07 01:45:15,674] Trial 2 finished with value: 1189.3425757743673 and parameters: {'num_layers': 13, 'num_nodes': 19, 'lr': 0.04222790494670366}. Best is trial 1 with value: 916.163.


Once the tuning job is finished, we can load the study as follows:

In [4]:
tutorial1 = get_study("tutorial1")

Let's take a quick look at the trials we ran:

In [5]:
tutorial1.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_lr,params_num_layers,params_num_nodes,state
0,0,NaN,2022-11-07 01:44:58,2022-11-07 01:45:05,0 days 00:00:07,0.002113,10,29,FAIL
1,1,916.163,2022-11-07 01:45:11,2022-11-07 01:45:13,0 days 00:00:02,0.031264,11,15,COMPLETE
2,2,1189.340,2022-11-07 01:45:13,2022-11-07 01:45:15,0 days 00:00:02,0.042228,13,19,COMPLETE


## 5. Parallel Tuning Jobs

Of course, for optimizing over a large hyperparameter search space, we will want to parallelize our jobs. Doing this with AmpOpt is easy: simply add the `jobs` argument. For example:

In [ ]:
ampopt.tune(
    study="tutorial2",
    trials=10,
    epochs=10,
    data="../data/oc20_3k_train.lmdb",
    params=format_params(
        dropout_rate=0.0,
        gamma=1.0,
    ),
    jobs=2,
)

## Reports and Summaries

To get a summary of all studies currently in the database, run

In [6]:
ampopt.view_studies()

Study tutorial1:
  Params:
    - lr
    - num_layers
    - num_nodes
  Best score: 916.163
  Num trials: 3


For a particular study, you can load it into memory and use `optuna.visualization.matplotlib` to easily visualise the study.

AmpOpt provides a single function for generating several interesting plots:

In [7]:
ampopt.generate_report("tutorial1")

Report directory C:\Users\ryanc\Desktop\Student Files\VIP\Alex-Repo\bdqm-hyperparam-tuning\report\tutorial1 already exists.


You can then view the generated plots in the `reports` folder of the project root.

Finally, perhaps you have run some experiments that aren't useful, and you'd like to clean up the list of studies. Run:

In [8]:
ampopt.delete_studies("tutorial1", "tutorial2")

Deleted study tutorial1.
